In [17]:
#kütüphaneler
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import IsolationForest

In [18]:
# verisetini hazırlama
column_names = ["Class","Alcohol","Malic acid","Ash","Alcalinity of ash","Magnesium",
                "Total phenols","Flavanoids","Nonflavanoid phenols","Proanthocyanins",
                "Color intensity","Hue","OD280/OD315 of diluted wines","Proline"]

path = "./vinedataset/wine.data" #verisetinin dosya yolu, kendi dosya yolunuzu giriniz.

df = pd.read_csv(path, names=column_names)

In [19]:
# NaN değerleri doldurma
imputer = KNNImputer(n_neighbors=5) # daha doğal biçimde median yerine local verileri korur.
df.iloc[:, 1:] = imputer.fit_transform(df.iloc[:, 1:])

In [20]:
# outliers değerleri temizleme 
iso_forest = IsolationForest(contamination=0.05, random_state=42) # daha esnek bir çözüm, sabit eşik değer yok.
outliers = iso_forest.fit_predict(df.iloc[:, 1:])
df = df[outliers == 1]  

In [21]:
# öznitelik ölçeklendirme
scaler = StandardScaler()
df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])

In [22]:
X = df.iloc[:, 1:] 
y = df['Class'] 

# eğitim test verilerini ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=60, stratify=y)

In [23]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42), # birden fazla ağaç ile overfitting riski azalır.
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42) # her adımda daha doğru sonuca yakınsayan ağaç eklenir.
}

In [24]:
# stratified fold ile validasyon
cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42) # class'lar arasındaki tutarsızlığı engellendi.

In [25]:
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"\n{name}:")
    print(f"Ortalama CV Doğruluk: {scores.mean():.4f}")
    print(f"Test Doğruluk: {accuracy_score(y_test, y_pred):.4f}")


Random Forest:
Ortalama CV Doğruluk: 0.9829
Test Doğruluk: 0.9804


AttributeError: 'super' object has no attribute '__sklearn_tags__'